
# Try out Bayesian update to environmental estimate


In [ ]:

# %%
#%%
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import xarray as xr
import arviz as az
import arviz.labels as azl
from hierarchical_normal_belk import hierarchical_normal
import itertools
#!! conda install -c conda-forge flox
import flox
from flox.xarray import xarray_reduce # useful in doing multiple coord groupings


In [ ]:
rng=np.random.Generator(np.random.PCG64(1234))


In [ ]:
size = 150 #size of grid
horz_offest = 50 #offset of grid from 0,0
mean_tempC_Km = 6.5/1000 #mean lapse rate
max_alt_Km = 13 #max altitude in km
#keep lat and long square for ease of matrixing
lat = np.arange(horz_offest, size)
long = np.arange(0, size - horz_offest)
alt = np.arange(0, max_alt_Km)*1000 #in meters


In [ ]:
# AR is auto-regressive, MA is moving average; will only use AR 
def sample_AR_signal(n_samples, corr, mu=0, sigma=1):
    assert 0 < corr < 1, "Auto-correlation must be between 0 and 1"
    burn_samples = 100
    n_samples=n_samples+burn_samples

    # Find out the offset `c` and the std of the white noise `sigma_e`
    # that produce a signal with the desired mean and variance.
    # See https://en.wikipedia.org/wiki/Autoregressive_model
    # under section "Example: An AR(1) process".
    c = mu * (1 - corr)
    sigma_e = np.sqrt((sigma ** 2) * (1 - corr ** 2))

    # Sample the auto-regressive process.
    signal = [c + np.random.normal(0, sigma_e)]
    for _ in range(1, n_samples):
        signal.append(c + corr * signal[-1] + np.random.normal(0, sigma_e))
    
    return np.array(signal[burn_samples:])
    
def compute_corr_lag_1(signal):
    return np.corrcoef(signal[:-1], signal[1:])[0][1]



# Baseline thermal along latitude




In [ ]:
base_sigma = .5
samp_lat_base = sample_AR_signal(size-horz_offest, 0.5, mu=2, sigma=base_sigma)
samp_lat= pd.DataFrame(samp_lat_base)
print(compute_corr_lag_1(samp_lat_base))
samp_lat.plot()


# Extend along longitude


In [ ]:
#use latitudes as mean for AR signal along longitudes
samp = sample_AR_signal(size-horz_offest, 0.75, mu=samp_lat, sigma=base_sigma)
samp = pd.DataFrame(samp[:, :, 0])
print(compute_corr_lag_1(samp.iloc[:,0]),compute_corr_lag_1(samp.iloc[0,:]))


In [ ]:

def plot_temperature_env(samp):
    x2, y2 = np.meshgrid(samp.index.values, samp.columns.values)
    plt.figure(figsize=(6,5))
    axes = plt.axes(projection='3d')
    axes.plot_surface(x2, y2,samp.values,cmap=cm.coolwarm,
                          linewidth=0, antialiased=False)
    axes.set_ylabel('Longitude')
    axes.set_xlabel('Latitude')
    axes.set_zlabel('Temperature')
    # keeps padding between figure elements
    plt.tight_layout()
    plt.show()

plot_temperature_env(samp)


# Add trend on top of the AR variation -- to baseline thermal


In [ ]:
# add MA
lat_inc_slope = 10/(size - horz_offest)  #deterministic slope of increase in temperature with latitude
lat_inc_max = lat_inc_slope *(size-horz_offest) 
long_inc_mu, long_inc_std = 20/(size - horz_offest), .1  #mean and std of increase in temperature with longitude

def add_inc_MA(size, horz_offest, sample_AR_signal, samp_lat, lat_inc_max, long_inc_mu, long_inc_std):
    lat_inc = np.linspace(0,lat_inc_max, len(samp_lat))
    sample_lat_inc = samp_lat[0] + lat_inc
    sample_lat_inc = pd.DataFrame(sample_lat_inc)
#sample_lat_inc.plot()

    samp_inc = sample_AR_signal(size-horz_offest, corr=0.5, mu=sample_lat_inc)
    long_inc = stats.norm.rvs(loc=long_inc_mu, scale=long_inc_std, size=(size-horz_offest,size-horz_offest), random_state=None)
    long_inc = np.cumsum(long_inc, axis=0)
    samp_inc = pd.DataFrame(samp_inc[:, :, 0]+long_inc)
    return samp_inc

samp_inc = add_inc_MA(size, horz_offest, sample_AR_signal, samp_lat, lat_inc_max, long_inc_mu, long_inc_std)
#bump temperature up to standard atmosphere
samp_inc=samp_inc+15-2


In [ ]:

plot_temperature_env(samp_inc)



# Extend into atmosphere


In [ ]:

#allow for inversion by having random lapse rate at diff altitudes
def add_altitude_effects(rng, samp_inc, mean_tempC_Km, max_alt_Km):
    tempC_Km = rng.normal(loc=mean_tempC_Km, scale=mean_tempC_Km/100, size=max_alt_Km)
# Temp at altitude = base temp - tempC_km * altitude
    temperature = ( [np.array(samp_inc) 
                 for _ in np.arange(max_alt_Km)]
               -np.broadcast_to(
    tempC_Km * alt, (size-horz_offest,size-horz_offest,max_alt_Km)
    ).T
)
    temperature = temperature.T
    return temperature

temp_3D = add_altitude_effects(rng, samp_inc, mean_tempC_Km, max_alt_Km)


In [ ]:

xr_temp_3D = xr.DataArray(temp_3D, dims=['lat', 'long', 'alt'], coords={'lat': lat, 'long': long, 'alt': alt})
fig = xr_temp_3D.plot.contourf(x='lat',y='long',col='alt', col_wrap=4,
                         robust=True)#, vmin=-90, vmax=32, levels=20)
plt.suptitle('Temperature at different altitudes', fontsize = 'xx-large',
             weight = 'extra bold')
plt.subplots_adjust(top=.92, right=.8, left=.05, bottom=.05)
plt.show()
 # plot temperature at different altitudes for lat 40 and long 140 and lat 80 and long 140 and lat 100 and long 100 

xr_temp_3D.sel(lat=40, long=140).plot(y='alt')
xr_temp_3D.sel(lat=80, long=140).plot(y='alt')
xr_temp_3D.sel(lat=100, long=100).plot(y='alt')
title = 'Temperature at different altitudes '
plt.title(title, fontsize = 'xx-large', weight = 'extra bold')
plt.legend(['lat 40 and long 140', 'lat 80 and long 140', 'lat 100 and long 100'])



xr_tempC_Km=  xr.DataArray(mean_tempC_Km, dims=['alt'], coords={'alt': alt})


In [ ]:
#check slopes

xr_4_slope = xr_temp_3D.sel(alt=10000,long=40)
slope_lat = xr_4_slope.diff('lat')/xr_4_slope.lat.diff('lat')
slope_lat.mean()
print(slope_lat.mean(),'\n---------------\n',lat_inc_slope, '\n\n')
xr_4_slope = xr_temp_3D.sel(alt=10000,lat=140)
slope_long = xr_4_slope.diff('long')/xr_4_slope.long.diff('long')
slope_long.mean()
print(slope_long.mean(),'\n---------------\n',long_inc_mu)


# Calculate pressure based on baseline temp field and assumed L; 


In [ ]:

#barometric formula
def add_barometric_effects(T = 288.15-273.15, L = 0.0065, H = 0,  P0 = 101_325.00, g0 = 9.80665, M = 0.0289644, R = 8.3144598):
    #barometric formula
    #P = P0 * (1 - L * H / T0) ^ (g0 * M / (R * L))
    #P = pressure
    #P0 = pressure at sea level = 101_325.00 Pa
    #L = temperature lapse rate = temperature lapse rate (K/m) in
    #H = altitude (m)
    #T0 = temperature at sea level = reference temperature (K)
    #g0 = gravitational acceleration = gravitational acceleration: 9.80665 m/s2
    #M = molar mass of air = molar mass of Earth's air: 0.0289644 kg/mol
    #R = gas constant = universal gas constant: 8.3144598 J/(mol·K)
    #L = temperature lapse rate
    #T = temperature
    if isinstance(T, xr.core.dataarray.DataArray):
        T0 = T.sel(alt=0)
        
    else:
        T0 = T[0]
        print('used t[0]')
        print(type(T))
    # not using temp at altitude directly so cna add 273.15 to just T0
    T0 = T0 +273.15

    #return P0 * (1 - L * H / (T0+273.15)) ** (g0 * M / (R * L))
    #return P0 * (1 + L * H / (T0+273.15)) ** (-g0 * M / (R * L))
    return P0 * (1 + (-L)*H/ T0) ** (-g0 * M / (R * (-L)))


'''pressure = add_barometric_effects(T = xr_temp_3D, 
                                 L = mean_tempC_Km, 
                                 H = xr_temp_3D.alt,  P0 = 101_325.00, g0 = 9.80665, M = 0.0289644, R = 8.3144598)'''
#put in ground temperature effects on pressure
P0_space = add_barometric_effects(T= xr_temp_3D, H=1 , P0 = 101_325.00, )

pressure = add_barometric_effects(T = xr_temp_3D, 
                                 L = mean_tempC_Km, 
                                 H = xr_temp_3D.alt,  P0 = P0_space, g0 = 9.80665, M = 0.0289644, R = 8.3144598)
pressure_backup = pressure.copy()


In [ ]:
#add noise to pressure
pressure=pressure_backup.pipe(lambda x: rng.normal(loc=x, scale=1000))-xr.zeros_like( pressure)

In [ ]:

xr_temp_pres = xr.merge(
    [xr_temp_3D.rename("Temperature"), 
     pressure.rename("Pressure")]
     )

xr_temp_pres.Pressure.plot.contourf(x='lat',y='long', col='alt', col_wrap=4,
                         robust=True, levels=20)
plt.suptitle('Pressure at different altitudes', fontsize = 'xx-large',
             weight = 'extra bold')
plt.subplots_adjust(top=.92, right=.8, left=.05, bottom=.05)


In [ ]:

#xr_temp_pres.mean(['lat', 'long']).plot.scatter(x='alt', y='Pressure' )
# add error bars to scatter plot
plt.errorbar(xr_temp_pres.mean(['lat', 'long']).alt, xr_temp_pres.mean(['lat', 'long']).Pressure,
                yerr=xr_temp_pres.std(['lat', 'long']).Pressure, fmt='o')

plt.title('Pressure at different altitudes', fontsize = 'xx-large',
                weight = 'extra bold')
plt.xlabel('Altitude (m)', fontsize = 'x-large',
                weight = 'extra bold')
plt.ylabel('Pressure (Pa)', fontsize = 'x-large',
                weight = 'extra bold')
plt.subplots_adjust(top=.92, right=.8, left=.05, bottom=.05)
plt.show()

# plot pressure at different altitudes for lat 40 and long 140 and lat 80 and long 140
xr_temp_pres.sel(lat=40, long=140).Pressure.plot.line(x='alt')
xr_temp_pres.sel(lat=80, long=140).Pressure.plot.line(x='alt')
xr_temp_pres.sel(lat=100, long=100).Pressure.plot.line(x='alt')
title = 'Pressure at different altitudes '
plt.title(title, fontsize = 'xx-large', weight = 'extra bold')
plt.legend(['lat 40 and long 140', 'lat 80 and long 140', 'lat 100 and long 100'])


# Make trajectory and get corresponding temp and pres


In [ ]:
# make Z = a function of time and  X = sin of time and y = cos of time
time = pd.to_datetime( np.arange(0, 3*60*60, 1), unit='s')
print(time)


In [ ]:

release_alt = 12_000 #Troposphere goes to about 12Km, thermal is about linear there
step_alt = 1
turn_rate = 3.14 #turns per hour
x = (np.sin((time.hour+time.minute/60 +time.second/3600)*(2*np.pi)*turn_rate) +1) * size/2.50 +30
y = (np.cos((time.hour+time.minute/60 +time.second/3600)*(2*np.pi)*turn_rate) +1 ) * size/2
#create samples from normal distribution and sort them
samples = stats.weibull_max.rvs(2.9, loc=0, scale=1, size=len(time), random_state=None)
samples.sort()
steps = samples/(samples.max()-samples.min()) /1.3  #normalize and shrink
steps = steps - steps.min() #shift to 0
 #smaller step per time
z = release_alt * (1- steps)

plt.plot(time, z)
plt.xlabel('Time')
plt.ylabel('Altitude')
plt.title('Altitude vs Time')
ax = plt.gca()
ax.set_ylim(0, 12000)
plt.show()
#plot 3d trajectory of z by x and y
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(x, y, z)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.set_zlim(0, 12000)
plt.title('3D Trajectory')
plt.show()


In [ ]:

#select from xarray the temperature at the pressure of the trajectory
xr_traj_env = xr.DataArray()

xr_x = xr.DataArray(x, dims=['time'], coords={'time': time})
xr_y = xr.DataArray(y, dims=['time'], coords={'time': time})
xr_z = xr.DataArray(z, dims=['time'], coords={'time': time})

xr_traj_env = xr_temp_pres.interp(lat=xr_x,long=xr_y,alt=xr_z)#, method='nearest')
xr_traj_env = xr_traj_env.interpolate_na(dim='time', method='linear', fill_value="extrapolate")
xr_traj_env.attrs =dict(units='seconds since 1970-01-01 00:00:00')
 # delay start of trajectory
xr_traj_env['time'] = xr_traj_env.time +pd.Timedelta(hours=.75)

xr_traj_env



# Put in Ballon data


In [ ]:

ballon_alt_samples = np.arange(start=0,stop=max_alt_Km*1000+1,step=500)
ballon_time = ballon_alt_samples/5
ballon_time = pd.to_datetime(  ballon_time, unit='s')
ballon_lat = [40, 80, 41, 139, 140]
ballon_long = [15, 100, 140, 139, 25]
launch_count = len(ballon_lat)
ballon_delay = [pd.Timedelta(hours=i)
                for i in [0,5,6.5,8,9]
                ]# 7*60*60 # 7 hrs later in seconds
launch_idx = np.arange(0,launch_count)
def ballon_release(xr_temp_pres, ballon_alt_samples, ballon_time, ballon_lat, ballon_long, ballon_delay, launch_idx):
    #ballon launch delay is in hours, will convert number to pd.Timedelta
    #ballon_delay = pd.Timedelta(hours=ballon_delay)
    coords={'launch':[launch_idx],'time':(('time'),ballon_time+ballon_delay)}
    xr_ballon_env = xr_temp_pres.interp(lat=
                                    xr.DataArray([[ballon_lat[launch_idx]]]*len(ballon_time), 
                                                 dims=['time','launch'],
                                                 coords=coords),
                                    long=
                                    xr.DataArray([[ballon_long[launch_idx]]]*len(ballon_time)
                                    ,
                                                 dims=['time','launch'],
                                                 coords=coords),
                                    alt=
                                    xr.DataArray([ballon_alt_samples],
                                                 dims=['launch','time'],
                                                 coords=coords),
                                    )
    xr_ballon_env= xr_ballon_env.interpolate_na(dim='time',method='linear', fill_value = 'extrapolate')
    xr_ballon_env.attrs =dict(units='seconds since 1970-01-01 00:00:00')
    return xr_ballon_env

#TODO: remove the 'launch' dimension from the ballon_release function, then do not squeeze it out
xr_ballon_env_lst = [ballon_release(xr_temp_pres, ballon_alt_samples, ballon_time,

                                 ballon_lat, ballon_long, ballon_delay[i], launch_idx=i).squeeze()
                                   for i in np.arange(launch_count)

                    ]
xr_ballon_env = xr.concat(xr_ballon_env_lst, dim='time')



xr_ballon_env


In [ ]:

xr_traj_env = xr.concat([xr_traj_env, 
                         xr_ballon_env.drop('launch')],
                           dim='time').sortby('time')
#plot 3d trajectory of z by x and y

def plot_traj_3d(xarray=None):
    xr_x= xarray
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(xr_x.lat,
            xr_x.long,
            xr_x.alt)
    ax.set_xlabel('Lat')
    ax.set_ylabel('Long')
    ax.set_zlabel('Alt')
    ax.set_zlim(0, xr_x.alt.max())
    plt.title('3D Trajectory')
    plt.show()

 

plot_traj_3d(xr_traj_env)

In [ ]:

xr_traj_env.Temperature.plot.scatter(x='time',color='black')
plt.suptitle('Temperature over time', fontsize = 'xx-large')
plt.show()
#make each dot black
xr_traj_env.Pressure.plot.scatter(x='time', y='Pressure', color='black')
plt.suptitle('pressure over time', fontsize = 'xx-large')
plt.show()



# downsample from xarray


In [ ]:
xr_traj_env.reset_coords(['lat','long','alt'], drop=False).resample(time='5min', restore_coord_dims=True
                                                                    ).mean().dropna(dim='time'
                                                                                    ).set_coords(['lat','long','alt']
                                                                                                 ).set_xindex(['alt','lat','long'])



In [ ]:

#must be a datetime index in xarray
# move xarray coordinate to variable
xr_traj_env_time = xr_traj_env.reset_coords(['lat','long','alt'], drop=False)
xr_traj_env_time = xr_traj_env_time.resample(time='5min', restore_coord_dims=True).mean().dropna(dim='time')
xr_traj_env_time_coords = xr_traj_env_time.copy(deep=True)
#Move variable to xarray coordinate multiindex
xr_traj_env_time= xr_traj_env.reset_coords(['lat','long','alt'], drop=False
                                           ).resample(time='5min', restore_coord_dims=True
                                                      ).mean().dropna(dim='time'
                                                                      ).set_coords(['lat','long','alt']
                                                                                   ).set_xindex(['alt','lat','long'])

'''#this duplicates the data :(
xr_traj_env_time = xr_traj_env_time.drop(['lat','long','alt'])
xr_traj_env_time = xr_traj_env_time.expand_dims({"lat":xr_traj_env_time_coords.lat.values, 
                              'long':xr_traj_env_time_coords.long.values, 
                              'alt':xr_traj_env_time_coords.alt.values}) '''


plot_traj_3d(xr_traj_env_time)
plt.show()

xr_traj_env_time_coords.Temperature.plot.scatter(x='time',color='black')
plt.suptitle('Temperature over time', fontsize = 'xx-large')
plt.show()
#make each dot black
xr_traj_env_time_coords.Pressure.plot.scatter(x='time', y='Pressure', color='black')
plt.suptitle('pressure over time', fontsize = 'xx-large')
plt.show()
# toDO CHANGE THIS VARIABLE BELOW HERE
#xarray make a multiindex of lat long alt and time

#grp_traj_env = 
# may be useful : xr_traj_env_time.stack(alt_lat_long_time=['alt','lat','long','time'],create_index=True)


In [ ]:
#xr_traj_env_time wher Temperature is between 26 and 27 degrees
xr_traj_env_time.sel( lat=40.0, long=15.0)#[0,:,:,0]


# # Using average values per time period along release; TODO: find more principled way to remove autocorrelation 



# # Model temp and pressure varying by altitude, lat, & long


In [ ]:
#standardize the data

def normalization(xarray=None):
    xr_x = xarray
    revert_min, revert_spread = xr_x.min(), xr_x.max()-xr_x.min()
    xr_x = (xr_x - revert_min) / revert_spread
    return xr_x, revert_min, revert_spread
def revert_normalization(xarray=None, revert_min=None, revert_spread=None):
    xr_x = xarray
    xr_x = xr_x * revert_spread + revert_min
    return xr_x

xr_traj_env_time['stz_lat'], sdz_min, stz_spread = normalization(xr_traj_env_time.lat)
xr_traj_env_time.attrs['sdz_lat_min'] =sdz_min.values
xr_traj_env_time.attrs['sdz_lat_spread']  =stz_spread.values
xr_traj_env_time['stz_long'], sdz_min, stz_spread = normalization(xr_traj_env_time.long)
xr_traj_env_time.attrs['sdz_long_min'] =sdz_min.values
xr_traj_env_time.attrs['sdz_long_spread']  =stz_spread.values
xr_traj_env_time['stz_alt'], sdz_min, stz_spread = normalization(xr_traj_env_time.alt)
xr_traj_env_time.attrs['sdz_alt_min'] =sdz_min.values
xr_traj_env_time.attrs['sdz_alt_spread']  =stz_spread.values

xr_traj_env_time

In [ ]:

coords={'alt_lat_long_time':
                      np.arange(xr_traj_env_time.sizes['time'], dtype=int)
                      }
with pm.Model(coords=coords) as thermal_pres:
    #Temp is in celcius
    
    Alt_ = pm.ConstantData('Altitude_m', xr_traj_env_time.alt.values,# use normal unstabdardized data
                                          dims='alt_lat_long_time' )
    Lat_ = pm.ConstantData('Latitude', xr_traj_env_time.stz_lat.values,# [bin_item.mid for bin_item in grp_traj_env.lat_bins.values],
                                        dims='alt_lat_long_time' )
    Long_ = pm.ConstantData('Longitude', xr_traj_env_time.stz_long.values,#[bin_item.mid for bin_item in grp_traj_env.long_bins.values],
                                          dims='alt_lat_long_time' )
    Temp_ = pm.ConstantData('Temperature_Samples', xr_traj_env_time.Temperature.values, dims='alt_lat_long_time' )
    Pres_ = pm.ConstantData('Pressure_Samples', xr_traj_env_time.Pressure.values, dims='alt_lat_long_time' )
    #prior on effect on temp (degC) of altitude and lat, long
    baseline_temp = pm.Normal('baseline_temp', mu=17+45, sigma=15)
    Alt_effect_temp = pm.Normal('Alt_effect_temp_Km', mu=-6, sigma=0.5)
    Lat_effect_temp = pm.Normal('Lat_effect_temp', mu=0, sigma=100/4)
    Long_effect_temp = pm.Normal('Long_effect_temp', mu=0, sigma=25/4)
    Lat_Long_effect_temp = 0  #pm.Normal('Lat_Long_effect_temp', mu=0, sigma=1)
    #prior on temp and pressure
    #TODO: PULL FROM DATABASE into a pm.Interpolated...maybe not: need relationship between data spreads?
    mu_t = pm.Deterministic('mu_t',
                               baseline_temp + 
                               Alt_effect_temp/1000 * Alt_ + 
                               Lat_effect_temp * Lat_ + 
                               Long_effect_temp * Long_ + 
                               Lat_Long_effect_temp * Lat_ * Long_, 
                               dims='alt_lat_long_time')
    #mu_t = hierarchical_normal('temperature_mean', mu= mu_mu_t, sigma = 2, dims='alt_lat_long_time')
    #mu_p = hierarchical_normal('pressure_mean', 
    P0 = pm.Normal('P0', mu=1, sigma=.01)*101_325.00 # lat/long influence on ground level temp captured in Temp_0
    g0 = 9.80665
    M = 0.0289644
    R = 8.3144598
    # NOTE: Temp_[0] is not the lowest altitude temperature, but the first temperature in the array
    Temp_0 = baseline_temp+ Lat_effect_temp * Lat_ + Long_effect_temp * Long_ # account for lat/long influence on ground level temp
    mu_p= pm.Deterministic('mu_p',P0 *  ((mu_t+273.15)/(Temp_0+273.15)) ** (-g0 * M / (R * (Alt_effect_temp/1000))), 
                                 dims='alt_lat_long_time')

    #add_barometric_effects = P0 * (T/T0) ** (-g0 * M / (R * L))
    #prior on error variation
    sigma_t=pm.Exponential('model_error_t', 1/5)
    sigma_p=pm.Exponential('model_error_p', 1/5000)
    #adjusted temp - normal dist error term
    obs_t = pm.Normal('obs_t', mu=mu_t, sigma=sigma_t,  observed = Temp_, dims='alt_lat_long_time')
    obs_p = pm.Normal('obs_p', mu=mu_p, sigma=sigma_p, observed = Pres_, dims='alt_lat_long_time')
    
pm.model_to_graphviz(thermal_pres)


## Examine the priors selected and their match to the observed

In [ ]:
with thermal_pres:
    idata = pm.sample_prior_predictive(1000)
az.plot_ppc(idata, group='prior', kind='cumulative')


## Run MCMC bayesian sampler

In [ ]:
with thermal_pres:
    idata.extend(pm.sample(1000, tune=1000, chains = 4, cores=1))
    # pymc sample posterior predictive check
    pm.sample_posterior_predictive(idata, extend_inferencedata=True)
    az.plot_ppc(idata, group='posterior', kind='cumulative')

In [ ]:
#As i am going to be messing with coordinates and attributes, i am going to make a copy of the data
idata2 = idata.copy()

In [ ]:
correctsignsize_mean_tempC_Km= -mean_tempC_Km*1000
az.plot_trace(idata2, lines = (('Alt_effect_temp_Km', {},[correctsignsize_mean_tempC_Km]),
                              #('Lat_effect_temp',{}, lat_inc_slope  ),
                              #('Long_effect_temp',{}, long_inc_mu  ), ))
                              #('Lat_effect_temp',{}, lat_inc_slope / xr_traj_env_time_scaled.attrs['sdz_lat_spread'] ),
                              #('Long_effect_temp',{}, long_inc_mu / xr_traj_env_time_scaled.attrs['sdz_long_spread'] ), ))
                              ('Lat_effect_temp',{}, lat_inc_slope  * xr_traj_env_time.attrs['sdz_lat_spread'] ),  
                              ('Long_effect_temp',{}, long_inc_mu * xr_traj_env_time.attrs['sdz_long_spread'] ), ))

plt.subplots_adjust (hspace=0.4)#, wspace=0.4)

In [ ]:
sortby_alt = xr.DataArray(idata2.constant_data.Altitude_m.values,
                                              coords={'alt_lat_long_time':idata2.constant_data.alt_lat_long_time.values})
idata2 = idata2.map(lambda ds, **kwarg: ds.sortby(**kwarg), groups= ["~sample_stats"],variables=sortby_alt, ascending=False)
idata2 = idata2.map(lambda ds, **kwarg: ds.assign_coords(**kwarg), 
           groups=['prior_predictive', 'posterior_predictive', 'observed_data'], 
           Altitude_m=(['alt_lat_long_time'], idata2.constant_data.Altitude_m.values))

idx_sortby_alt = idata2.constant_data.alt_lat_long_time.values


In [ ]:
#xarray filter by values - used in grouping graphs
lat_min = idata2.constant_data.Latitude.min()
lat_max = idata2.constant_data.Latitude.max()
long_min = idata2.constant_data.Longitude.min()
long_max = idata2.constant_data.Longitude.max()
lat_mid = (lat_min + lat_max)/2
long_mid = (long_min + long_max)/2

idx_north = idata2.constant_data.where(idata2.constant_data.Latitude>lat_mid, drop=True).alt_lat_long_time.values
idx_south = idata2.constant_data.where(idata2.constant_data.Latitude<lat_mid, drop=True).alt_lat_long_time.values
idx_east = idata2.constant_data.where(idata2.constant_data.Longitude>long_mid, drop=True).alt_lat_long_time.values
idx_west = idata2.constant_data.where(idata2.constant_data.Longitude<long_mid, drop=True).alt_lat_long_time.values


In [ ]:
# creating class for labeling the altitude instead of variable index
class DimCoordLabeller_alt(azl.BaseLabeller):
    """WIP."""
    def __init__(self, coords_ds):
        self.coords_ds = xr.Dataset(coords)

    def dim_coord_to_str(self, dim, coord_val, coord_idx):
        """WIP."""#format decimals in f statement
        temp =  self.coords_ds.sel(pointwise_sel=coord_val).items()
        temp = [(v.values) for _,v in temp][0]
        return f"{temp:.2f}" 
    
coords = {
    'alt_lat_long_time': xr.DataArray(
        idata2.constant_data.Altitude_m.values, 
        dims=['pointwise_sel'],coords={'pointwise_sel': idata2.constant_data.alt_lat_long_time.values}
    )
}        
labeller = DimCoordLabeller_alt(coords)


In [ ]:
#figures with lat in coulmns and long in rows
fig, ax = plt.subplots(2, 2, figsize=(10, 10))
ax = ax.flatten()
for i, ((N_S_label,N_S_idx), (E_W_label,E_W_idx)) in enumerate([[i,j] 
                                  for i in [('North',idx_north),('South',idx_south)] 
                                  for j in [('West',idx_west), ('East',idx_east)]]):
    print(N_S_label,E_W_label)
    ax[i].set_title(f'Lat: {N_S_label} Long: {E_W_label}')
    idx = np.intersect1d(N_S_idx,E_W_idx)
    az.plot_forest(idata2.sel(alt_lat_long_time=[x for x in idx_sortby_alt
                                                 if np.any(np.isin(idx,x))]),
                   var_names=['mu_t'],
                   kind='ridgeplot', 
                   combined=True, ax= ax[i],
                   labeller=labeller
                   )
    #align the y axis
    #ax[i].set_ylim(0, 10000)
    ax[i].set_xlim(-60, 120)
    ax[i].grid()


In [ ]:
az.plot_forest(idata2.sel( alt_lat_long_time=idx_sortby_alt), var_names=['mu_t'],kind='ridgeplot', combined=True,labeller=labeller)
az.plot_forest(idata2.sel( alt_lat_long_time=idx_sortby_alt), var_names=['mu_p'],kind='ridgeplot', combined=True,labeller=labeller)


In [ ]:
idata_pred_Alt = idata2.map( lambda ds, **kwarg: 
                                   ds.swap_dims(**kwarg).drop('alt_lat_long_time'),#.drop_indexes('Altitude_m'), 
                                   groups=['observed','predictive'], filter_groups='like', 
                                   alt_lat_long_time='Altitude_m')
idata_pred_Alt
az.plot_dist_comparison(idata_pred_Alt, 
                        kind='observed', 
                        #labeller=labeller, 
                        )
plt.show()

In [ ]:
az.plot_dist_comparison(idata_pred_Alt, 
                        kind='observed', 
                        var_names=['obs_p']
                        #labeller=labeller, 
                        )
plt.show()